# Graphing Code to Load Data and Plot It Live

## Imports

In [ ]:
import plotly as py
import plotly.tools as tls
from plotly import graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly.subplots import make_subplots
import plotly.express as px
import numpy as np
import pandas as pd
import datetime
import json
import dash
from dash.dependencies import Output, Input
import dash_core_components as dcc
import dash_html_components as html
import Processing_Functions_Tracking
from ast import literal_eval

## Initialising Dash and Plotly

In [ ]:
app = dash.Dash(__name__)

#Plotly offline mode
init_notebook_mode(connected=True)

timer = 0

In [ ]:
#Function to check if any of the items contain an array
def array_check(item,timer):
    item = item[0]
    print(item)
    if(isinstance(item, str)):
        print(item)
        item = literal_eval(item)
        return round(item[timer],2)
    elif(isinstance(item, list)):

        return round(item[-1],2)
    else:

        return round(float(item),2)

# Loading Data

In [ ]:
#code = "G10392.0"
#path = "Experiments/100 Centre" + "/100 Robots/" + code + "/"
#path = path = "../Experiments/Detecting Dust Devils - Passive Survey 1. Final Deployment Sweep with Dust Devils/50 Robots/"
"""global count
global path"""

path = "../Experiments/Detecting Dust Devils - Testing Tracking Functionality by using Clustering Algorithm And No Broadcasting G. Checking Healing of 100 Robots/100 Robots/"


store_robots = np.load(path + "Robots.npy")

with open(path + "dust.txt", "r") as f:
    dust = json.load(f)


constants = pd.read_excel(path + "Constants.xlsx", index_col=0)
min_neighbours = np.load(path + "Minimum Distance to Neighbours.npy")
cluster_average = np.load(path + "Cluster Average.npy")
total_collision = np.load(path + "Measurement Events Count.npy")
total_detection = np.load(path + "Number of Dust Devils Detected.npy")
total_dust = np.load(path + "Number of Dust Devils Generated.npy")
robot_number = constants.loc["Number of Robots"]
R = constants.loc["Communication Range"]
max_speed = constants.loc["Max Speed"]
G = constants.loc["G"]
power = constants.loc["Power"]
max_force = constants.loc["Max Force"]


In [ ]:
"""path1 = "../Experiments/Detecting Dust Devils - Testing Tracking Functionality by using Clustering Algorithm And No Broadcasting G. Checking Healing of 100 Robots/100 Robots/"
path2  = "../Experiments/Detecting Dust Devils - Testing Tracking Functionality by using Clustering Algorithm And Broadcasting G. Checking Healing of 100 Robots/100 Robots/""
load(path)"""

In [ ]:

#the update layout function
def update_layout():
    #contains the graph, the interval timing and the live updating table
    app.layout = html.Div(children=[html.Div(
    [
    dcc.Graph(id = 'live_graph'),
    dcc.Interval(
        id = 'graph-update',
        interval = 100, #milliseconds
        n_intervals = 0
        ),
    ],style = {'display': 'inline-block'} ),
        html.Div(
            [
                dcc.Graph(id = 'live_table'),
                dcc.Interval(
                id = 'variable-update',
                interval = 100, #milliseconds
                n_intervals = 0
                )    
                 ],
        style = {'display': 'inline-block','vertical-align': 'top' }    
    )],style={'width': '100%', 'display': 'inline-block'})

update_layout()

In [ ]:
#callback is the updating process
@app.callback(Output('live_graph','figure'),
              [Input('graph-update', 'n_intervals')])

#the actual update for the graph
def update_graph(n):
    #initialising global variables
    global timer
    
    if(timer>4999):
        timer = 0
    position_robot = store_robots[:,:,timer]
    position_dust = dust[timer]
    x_dust = position_dust[0]
    y_dust = position_dust[1]
    if(not x_dust or not y_dust):
        x_dust = [null] 
        y_dust = [null] 
    #creating scatter plot of robots
    data = go.Scatter(
        x=list(position_robot[0]),
        y=list(position_robot[1]),
        name = 'Robots',
        mode = 'markers',
    )
    
    #creating the plotly figure with the robot data
    fig = go.Figure(
        { "data": data, "layout": go.Layout(yaxis=dict(range=[-500, 500]),xaxis = dict(range=[-500,500]))
        }
    )

    fig.add_trace(go.Scatter(
          x= position_dust[0],
          y= position_dust[1],
          name='Dust Devil',
          mode = 'markers',
          marker_color='rgba(255, 182, 193, .9)',
      ))
    
    #updating layout with circles and different formatting'''
    fig.update_layout(title="<b>Tracking Algorithm Demonstrating Self Healing Swarm Properties with No G Broadcast</b>",
    title_x=0.5,
    xaxis_title="X Position (m)",
    yaxis_title="Y Position (m)",
    margin=dict(
        t=50, # top margin: 30px, you want to leave around 30 pixels to
              # display the modebar above the graph.
         # bottom margin: 10px
        l=10, # left margin: 10px
        r=10, # right margin: 10px
    ),
    height=900,width=1150,
    xaxis = dict(
        tickmode = 'linear',
        tick0 = 0,
        dtick = 50
    ),
                      yaxis = dict(
        tickmode = 'linear',
        tick0 = 0,
        dtick = 50
    )
                     )
    fig.update_xaxes(showline=True, linewidth=2, linecolor='black',mirror=True)
    fig.update_yaxes(showline=True, linewidth=2, linecolor='black',mirror=True)
    #incrementing timer
    timer = timer + 5
    
    #updating the layout
    update_layout()
    return fig

#app callback for the table
@app.callback(Output('live_table','figure'),
              [Input('variable-update', 'n_intervals')])

def update_variables(n):
    fig=Processing_Functions_Tracking.table_figure_dash(timer,robot_number,R,G,power,max_force,max_speed,min_neighbours[timer],cluster_average[timer])
         #fig.layout['template']['data']['table'][0]['header']['fill']['color']='#636EFA'
    fig.update_layout(width = 650, height = 1500,margin=dict(
    t=260, # top margin: 30px, you want to leave around 30 pixels to
              # display the modebar above the graph.
        b=10, # bottom margin: 10px
        l=10, # left margin: 10px
        r=10,
    ))
    fig.update_yaxes(showline=True, linewidth=2, linecolor='black', mirror=True)
    fig.update_xaxes(showline=True, linewidth=2, linecolor='black', mirror=True)
    return fig

In [ ]:
#runs by default
if __name__ == '__main__':
    #running the app server for the simulator
    app.run_server(debug=False, host = '127.0.0.1')
    #app.run_server(debug=False,port=8080,host='0.0.0.0')